In [1]:
project = "datascience_247money_scorecard"

In [2]:
import pandas as pd
import numpy as np
import os


from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import MLClient
from datascience_core.authentications import DataLakeAuthentication, DWHAuthentication, AzureAuthenticationSingleton
from azure.core.credentials import TokenCredential, AzureSasCredential, AccessToken
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential


from azure.storage.filedatalake import DataLakeServiceClient
from azure.storage.blob import StorageStreamDownloader
from datascience_core.data_retrieval._save_data import BlobLocation
from datascience_core.data_retrieval import DataLakeCSVLoader
from datascience_core import cfg

Class SystemCreatedStorageAccount: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SystemCreatedAcrAccount: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [3]:
credentials = DWHAuthentication().get_credentials()
credentials=DataLakeAuthentication().get_credentials()

Creating authenticator


## Connect to ML Studio

In [4]:
ml_client = MLClient(
    credential=credentials, 
    subscription_id="6cbe45a0-6565-4c4a-b1a9-0929f276bbcd",
    resource_group_name='rg-data-science-dev',
    workspace_name='mlw-ml-dev')

Class RegistryOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [13]:
assets = []
asset_list = ml_client.data.list()#(name='test',label='latest')
[assets.append(x.name) for x in asset_list]
assets

['live_vehicle_m0_Sep.csv',
 'combined_m0_2months.csv',
 'reduced_catalogue.csv',
 'click_data.csv',
 'catalogue.csv',
 'catalogue',
 'click_data',
 'reduced_catalogue',
 'THIS_IS_TEST_DATA',
 'scorecard_cra_dataset',
 'test']

In [3]:
import uuid

file_uuid = uuid.uuid4()

str(file_uuid)

'71dd5e07-989a-4c0f-94a4-b05aacd5892f'

## Register a dataset

In [12]:
scorecard_dataset_path= "abfss://projects@ds247dldev.dfs.core.windows.net/datascience_247money_scorecard/Datasets/scorecard_cra_dataset/scorecard_cra_dataset.csv"

my_data = Data(
    path=scorecard_dataset_path,
    type=AssetTypes.URI_FILE,
    description="247money scorecard development dataset",
    name="scorecard_cra_dataset",
    version="2"
)

ml_client.data.create_or_update(my_data)

Data({'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_file', 'is_anonymous': False, 'auto_increment_version': False, 'name': 'scorecard_cra_dataset', 'description': '247money scorecard development dataset', 'tags': {}, 'properties': {}, 'id': '/subscriptions/6cbe45a0-6565-4c4a-b1a9-0929f276bbcd/resourceGroups/rg-data-science-dev/providers/Microsoft.MachineLearningServices/workspaces/mlw-ml-dev/data/scorecard_cra_dataset/versions/2', 'Resource__source_path': None, 'base_path': 'c:\\Users\\chris.pickford\\OneDrive - CarFinance 247 Ltd\\Documents\\Coding\\Python\\Projects\\datascience_core\\datascience_core\\jupytertools', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x0000023D5B8BB7F0>, 'serialize': <msrest.serialization.Serializer object at 0x0000023D5CBBFB80>, 'version': '2', 'latest_version': None, 'path': 'abfss://projects@ds247dldev.dfs.core.windows.net/datascience_247money_scorecard/Datasets/scorecard_cra_da

## List datasets in the datalake

In [5]:

project = 'datascience_247money_scorecard'
authentication = DataLakeAuthentication()
credential = authentication.get_azure_credentials()
data_lake_name = cfg["storage"]["data_lake"]
account_url = f"https://{data_lake_name}.dfs.core.windows.net/"
datalake_service = DataLakeServiceClient(
    account_url=account_url, credential=credential
)
blob = BlobLocation("ds247dldev", "projects", project+"/Datasets/", "")




Creating authenticator


In [6]:
file_client = datalake_service.get_file_client(blob.get_container(), blob.get_filepath())

In [7]:
file_system_client = datalake_service.get_file_system_client(file_system=blob.get_container())

In [8]:
datasets_path = []
print(blob.get_container())
paths = file_system_client.get_paths(path=blob.get_filepath())
for path in paths:
    temp = os.sep.join(os.path.normpath(path.name).split(os.sep)[:-1])
    datasets_path.append(os.sep.join(os.path.normpath(temp).split(os.sep)[-1:]))


datasets_path = list(np.unique(datasets_path))
datasets_path.remove('Datasets')
datasets_path

projects


['scorecard_cra_dataset']

## Pull registered dataset

In [120]:


asset_to_get= f"abfss://{blob.get_container()}@ds247dldev.dfs.core.windows.net/{blob.get_filepath()}/{datasets[0]}"
print(asset_to_get)

filedataset_asset = ml_client.data.get(name=datasets_path[0],label='latest')




abfss://projects@ds247dldev.dfs.core.windows.net/datascience_247money_scorecard/Datasets//scorecard_cra_dataset


In [122]:
filedataset_asset.path

'abfss://test@ds247dldev.dfs.core.windows.net/scorecard_dataset_build/outputs/AD4_cra_dev.csv'

In [130]:
initial_split = scorecard_dataset_path.split(sep="/")
initial_split.remove("abfss:")
initial_split.remove("")
initial_split

container = [x.split("@")[0] for x in initial_split if "@" in x][0]
account = [x.split("@")[1].split(".")[0] for x in initial_split if "@" in x][0]
path_name = [x for x in initial_split if "@" not in x]

file_name = path_name[-1]
path_name.remove(file_name)
path = ""
for component in path_name:
    path+=component+"/"

print(account)
print(container)
print(path)
print(file_name)

ds247dldev
test
scorecard_dataset_build/outputs/
AD4_cra_dev.csv


In [127]:
scorecard_dataset_path

'abfss://test@ds247dldev.dfs.core.windows.net/scorecard_dataset_build/outputs/AD4_cra_dev.csv'

In [10]:
auth = DataLakeAuthentication()
csv_loader = DataLakeCSVLoader(authentication=auth)

# Testing

In [7]:
from datascience_core.data_retrieval import ProjectDatasetLoader,DatasetUriGenerator

In [13]:
project = "datascience_247money_scorecard"
data_loader = ProjectDatasetLoader(project)
datasets=data_loader.load_dataset(version=2)

pulling dataset scorecard_cra_dataset.csv


c:\Users\chris.pickford\Anaconda3\envs\AD3_env\lib\site-packages\datascience_core\data_retrieval\_load_data.py:69: DtypeWarning: Columns (6,7,9,10,11,12,13,18,19,20,21,22,23,24,26,27,28,30,31,32,33,34,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,65,66,69,70,71,74,75,76,77,78,79,80,82,84,85,86,87,88,92,93,94,95,97,98,99,100,101,102,103,104,105,111,113,114,116,118,119,120,122,123,125,128,130,132,133,136,137,139,140,141,142,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,170,173,174,175,176,177,178,180,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,198,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,218,219,220,221,223,224,225,226,228,229,230,231,232,235,236,237,238,239,240,241,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,271,275,278,279,280,282,284,286,287,288,290,291,294,295,298,300,301,302,303,304,305,306,307,308,309,310,311,312,

In [14]:
datasets.keys()

dict_keys(['scorecard_cra_dataset.csv'])

In [12]:
df = datasets['AD4_cra_dev.csv']
df.head(5)

,Unnamed: 0,ApplicationId,_c0,MonthsFromEpoch,RawResponseId,BSB AF,BSB BE,BSB BIC,BSB BLC,BSB CF,...,App DistrictResidential,App EmploymentStatusName,App Occupation,App NetMonthlyIncome,App YearsAtEmploymentAddress,App MonthsAtEmploymentAddress,App PostcodeEmployment,App PostcodeDistrictEmployment,App DistrictEmployment,Target
0,0,8553380,3,1428.0,491283.0,{ND},{ND},0.0,{ND},{ND},...,NaN,Employed Full Time,Customer service,1500.0,17.0,3.0,NE331SJ,NE33,NaN,0
1,1,8553158,7,1428.0,500085.0,{ND},{ND},0.0,{ND},{ND},...,NaN,Employed Full Time,Lift engineer,3000.0,12.0,0.0,CV34FJ,CV3,NaN,0
2,2,8554384,9,1428.0,503517.0,{ND},{ND},0.0,{ND},{ND},...,NaN,Employed Full Time,CEO,2996.0,12.0,8.0,EH44UT,EH4,NaN,0
3,3,8552996,12,1428.0,516659.0,{ND},{ND},0.0,{ND},{ND},...,NaN,Employed Full Time,Project coordinator,1500.0,2.0,0.0,M204BX,M20,NaN,0
4,4,8527682,23,1428.0,518195.0,{ND},10789,0.0,{ND},{ND},...,NaN,Employed Full Time,Coach driver,1500.0,0.0,3.0,NG139HP,NG13,NaN,0


In [25]:
uri="abfss://@account.dfs.core.windows.net/dir1/dir2/dir3/file.csv"
initial_split = uri.split(sep="/")
initial_split.remove("abfss:")
initial_split.remove("")
print(initial_split)
container = [x.split("@")[0] for x in initial_split if "@" in x][0]
account = [x.split("@")[1].split(".")[0] for x in initial_split if "@" in x][0]

['@account.dfs.core.windows.net', 'dir1', 'dir2', 'dir3', 'file.csv']


In [3]:
data_loader.testing_paths


<iterator object azure.core.paging.ItemPaged at 0x29751810730>

In [4]:
EXAMPLE_PATHS=[]
for path in data_loader.testing_paths:
   print(path)

EXAMPLE_PATHS

{'name': 'datascience_247money_scorecard/Datasets/scorecard_cra_dataset', 'owner': '$superuser', 'group': '$superuser', 'permissions': 'rwxr-x---', 'last_modified': datetime.datetime(2023, 1, 10, 15, 15, 47), 'is_directory': True, 'etag': '0x8DAF31D8D7849F2', 'content_length': 0, 'creation_time': datetime.datetime(2023, 1, 10, 15, 15, 47, 941835, tzinfo=datetime.timezone.utc), 'expiry_time': None, 'encryption_scope': None}
{'name': 'datascience_247money_scorecard/Datasets/scorecard_cra_dataset/scorecard_cra_dataset.csv', 'owner': '$superuser', 'group': '$superuser', 'permissions': 'rw-r-----', 'last_modified': datetime.datetime(2023, 1, 11, 8, 57, 37), 'is_directory': False, 'etag': '0x8DAF3B1E316CBC5', 'content_length': 19308411, 'creation_time': datetime.datetime(2023, 1, 11, 8, 57, 37, 102945, tzinfo=datetime.timezone.utc), 'expiry_time': None, 'encryption_scope': None}


[]

In [5]:
import datetime
EXAMPLE_DATA = [{'name': 'datascience_247money_scorecard/Datasets/scorecard_cra_dataset', 'owner': '$superuser', 'group': '$superuser', 'permissions': 'rwxr-x---', 'last_modified': datetime.datetime(2023, 1, 10, 15, 15, 47), 'is_directory': True, 'etag': '0x8DAF31D8D7849F2', 'content_length': 0, 'creation_time': datetime.datetime(2023, 1, 10, 15, 15, 47, 941835, tzinfo=datetime.timezone.utc), 'expiry_time': None, 'encryption_scope': None},
{'name': 'datascience_247money_scorecard/Datasets/scorecard_cra_dataset/scorecard_cra_dataset.csv', 'owner': '$superuser', 'group': '$superuser', 'permissions': 'rw-r-----', 'last_modified': datetime.datetime(2023, 1, 11, 8, 57, 37), 'is_directory': False, 'etag': '0x8DAF3B1E316CBC5', 'content_length': 19308411, 'creation_time': datetime.datetime(2023, 1, 11, 8, 57, 37, 102945, tzinfo=datetime.timezone.utc), 'expiry_time': None, 'encryption_scope': None}]

In [6]:
for path in EXAMPLE_DATA:
    path.name

AttributeError: 'dict' object has no attribute 'name'

In [6]:
scorecard_dataset_path="abfss://projects@ds247dldev.dfs.core.windows.net/datascience_247money_scorecard/Datasets/scorecard_cra_dataset"
EXAMPLE_DATA = Data(
    path=scorecard_dataset_path,
    type=AssetTypes.URI_FILE,
    description="247money scorecard development dataset",
    name="file",
    version="2"
)

EXAMPLE_DATA

Data({'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_file', 'is_anonymous': False, 'auto_increment_version': False, 'name': 'file', 'description': '247money scorecard development dataset', 'tags': {}, 'properties': {}, 'id': None, 'Resource__source_path': None, 'base_path': 'c:\\Users\\chris.pickford\\OneDrive - CarFinance 247 Ltd\\Documents\\Coding\\Python\\Projects\\datascience_core\\datascience_core\\jupytertools', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x0000020DEEC69AF0>, 'version': '2', 'latest_version': None, 'path': 'abfss://projects@ds247dldev.dfs.core.windows.net/datascience_247money_scorecard/Datasets/scorecard_cra_dataset', 'datastore': None})

In [7]:

project_assets = []
paths =[EXAMPLE_DATA]
for path in paths:
    temp = os.sep.join(os.path.normpath(path.name).split(os.sep)[:-1])
    project_assets.append(
        os.sep.join(os.path.normpath(temp).split(os.sep)[-1:])
    )

project_assets = list(np.unique(project_assets))
project_assets.remove("Datasets")

ValueError: list.remove(x): x not in list

In [8]:
os.sep.join(os.path.normpath(path.name).split(os.sep)[:-1])

''

In [19]:
import pandas as pd
from datascience_core.data_retrieval import TierMapper
tier_boundaries = {
    'Tier1': 0.19,
    'Tier2': 0.26,
    'Tier3': 0.32,
    'Tier4': 0.43,
    'Tier5': 0.48,
    'Tier6': 0.52
    }
mapper = TierMapper(tier_boundaries,tier_boundaries)


tier_boundaries = {
    'Tier1': 0.19,
    'Tier2': 0.26,
    'Tier3': 0.32,
    'Tier4': 0.43,
    'Tier5': 0.48,
    'Tier6': 0.52
    }
mapper = TierMapper(tier_boundaries,tier_boundaries)

scores = {}
AD_list = []
unmapped_list=[]

for i in range(101):
    i /= 100
    AD_list.append(i)
    unmapped_list.append(mapper._unmap_score(AD_score= i,raw_score_boundaries_dict=tier_boundaries))

scores['AD'] = AD_list
scores['raw'] = unmapped_list

scores = pd.DataFrame(scores)

In [21]:
AD_score = 0.83
mapper._unmap_score(AD_score= AD_score,raw_score_boundaries_dict=tier_boundaries)

0.6736

In [22]:
scores = {}
AD_list = []
unmapped_list=[]

for i in range(101):
    i /= 100
    AD_list.append(i)
    unmapped_list.append(mapper._unmap_score(AD_score= i,raw_score_boundaries_dict=tier_boundaries))

scores['AD'] = AD_list
scores['raw'] = unmapped_list

scores = pd.DataFrame(scores)

In [38]:
#scores.set_index('AD',inplace=True)

In [23]:
scores.iloc[1:50]

,AD,raw
1,0.01,0.007600
2,0.02,0.015200
3,0.03,0.022800
4,0.04,0.030400
5,0.05,0.038000
6,0.06,0.045600
7,0.07,0.053200
8,0.08,0.060800
9,0.09,0.068400
10,0.10,0.076000


In [7]:
abf='abfss://projects@ds247dldev.dfs.core.windows.net/datascience_247money_scorecard/Datasets/scorecard_cra_dataset/scorecard_cra_dataset.csv'

uri = 'https://ds247dldev.blob.core.windows.net/projects/test/Datasets/test/6f080ac3-abef-4e7a-88de-4ec6ebb6d569'

In [17]:
initial_split = uri.split(sep="/")
initial_split.remove("https:")
initial_split.remove("")

account = [x.split(".")[0] for x in initial_split if "." in x][0]
if len(account) == 0:
    raise
container = initial_split[1]
path = [x for x in initial_split if account+".blob.core.windows.net" not in x]
file_name = path[-1]
path.remove(file_name)
path.remove(container)
# file_name = file_name.split(".")[0]
file_path = ""
for component in path:
    file_path += component + "/"
path_components = {
    "account": account,
    "container": container,
    "file_path": file_path[:-1],
    "file_name": file_name,
}
return path_components

{'account': 'ds247dldev', 'container': 'projects', 'file_path': 'test/Datasets/test', 'file_name': '6f080ac3-abef-4e7a-88de-4ec6ebb6d569'}


In [ ]:
initial_split = uri.split(sep="/")
initial_split.remove("https:")
initial_split.remove("")
initial_split

container = [x.split("@")[0] for x in initial_split if "@" in x][0]
if len(container) == 0:
    raise
account = [x.split("@")[1].split(".")[0] for x in initial_split if "@" in x][0]
if len(account) == 0:
    raise
path = [x for x in initial_split if "@" not in x]

file_name = path[-1]
path.remove(file_name)
# file_name = file_name.split(".")[0]
file_path = ""
for component in path:
    file_path += component + "/"

path_components = {
    "account": account,
    "container": container,
    "file_path": file_path[:-1],
    "file_name": file_name,
}


In [6]:
from datascience_core.data_retrieval import ProjectDatasetManager
import pandas as pd

In [2]:
dataset_manager = ProjectDatasetManager(project='test')

Class RegistryOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Creating authenticator


In [7]:
dataset1 = pd.DataFrame({'col1':[0,1,2,3,4,5],'col5':[0,1,2,3,4,5]})
dataset2 = pd.DataFrame({'col1':[0,1,2,3,4,5],'col5':[0,1,2,3,4,5]})

dataset_manager.register_dataset(registered_dataset_name="test2",data=dataset1)
dataset_manager.register_dataset(registered_dataset_name="test3",data=dataset2)

In [10]:
data_dict = dataset_manager.load_dataset()

pulling dataset test
pulling dataset test2
pulling dataset test3


In [12]:
data_dict['test']

,col1,col2
0,0,0
1,1,1
2,2,2
3,3,3
4,4,4
5,5,5


In [13]:
data = {"col1": [0, 1, 2, 3, 4, 5], "col2": [0, 1, 2, 3, 4, 5]}
data = pd.DataFrame(data)

In [15]:
pd.testing.assert_frame_equal(data_dict['test'],data)

In [16]:
asset_paths = [{'test': 'https://ds247dldev.blob.core.windows.net/projects/test/Datasets/test/dfcd18fc-02c6-42da-b7ca-2f9276d7dcbf'}]

In [19]:
list(asset_paths[0].keys())[0]

'test'